In [1]:
###### %matplotlib inline
# %config InlineBackend.figure_format = 'retina'

# convolutional autoencoder in keras

import os
#os.environ["KERAS_BACKEND"] = "tensorflow"

from __future__ import print_function
from keras.models import Model
from keras.layers import Dense, Input, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, BatchNormalization
from keras.datasets import mnist
from keras.regularizers import l1
from keras.optimizers import Adam
from keras import backend as K

import matplotlib.pyplot as plt
# %matplotlib inline

from keras.datasets import mnist 

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

def own_kullback_leibler_divergence(y_true, y_pred):
    y_true = K.clip(y_true, K.epsilon(), 1)
    y_pred = K.clip(y_pred, K.epsilon(), 1)
    return K.sum(y_true * K.log(y_true / y_pred), axis=-1)

def own_mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

def own_loss(y_true, y_pred):
    return 1.0*own_kullback_leibler_divergence(y_true, y_pred)+own_mean_squared_error(y_true, y_pred)

# utility function for showing images
def show_imgs(x_test, decoded_imgs, n=10):
    plt.figure(figsize=(20, 4))
    for i in range(n):
        ax = plt.subplot(2, n, i+1)
        plt.imshow(x_test[i].reshape(28,28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        if decoded_imgs is not None:
            ax = plt.subplot(2, n, i+ 1 +n)
            plt.imshow(decoded_imgs[i].reshape(28,28))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
    plt.show()

### Create input

(x_all, y_all), (x_all_test, y_all_test) = mnist.load_data()

x_all = x_all.astype('float32')/255. # 0-1.に変換
x_all_test = x_all_test.astype('float32')/255.

###

x_9_list=[]
for i in range(len(y_all)):
    if (y_all[i] == 9):
        x_9_list.append(x_all[i])
x_9=np.asarray(x_9_list[:5000])
x_9_list_test=[]
for i in range(len(y_all_test)):
    if (y_all_test[i] == 9):
        x_9_list_test.append(x_all_test[i])
x_9_test=np.asarray(x_9_list_test[:500])

###

x_6_list=[]
for i in range(len(y_all)):
    if (y_all[i] == 6):
        x_6_list.append(x_all[i])
x_6=np.asarray(x_6_list[:5000])
x_6_list_test=[]
for i in range(len(y_all_test)):
    if (y_all_test[i] == 6):
        x_6_list_test.append(x_all_test[i])
x_6_test=np.asarray(x_6_list_test[:500])

###

x_8_list=[]
for i in range(len(y_all)):
    if (y_all[i] == 8):
        x_8_list.append(x_all[i])
x_8=np.asarray(x_8_list[:5000])
x_8_list_test=[]
for i in range(len(y_all_test)):
    if (y_all_test[i] == 8):
        x_8_list_test.append(x_all_test[i])
x_8_test=np.asarray(x_8_list_test[:500])

###

x_9 = np.reshape(x_9, (len(x_9), 28, 28, 1))
x_9_test = np.reshape(x_9_test, (len(x_9_test), 28, 28, 1))

x_6 = np.reshape(x_6, (len(x_6), 28, 28, 1))
x_6_test = np.reshape(x_6_test, (len(x_6_test), 28, 28, 1))

x_8 = np.reshape(x_8, (len(x_8), 28, 28, 1))
x_8_test = np.reshape(x_8_test, (len(x_8_test), 28, 28, 1))

print(x_9.shape,x_6.shape,x_8.shape)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(5000, 28, 28, 1) (5000, 28, 28, 1) (5000, 28, 28, 1)


In [2]:
### pretrain1
    
input_img = Input(shape=(28, 28,1)) # 1ch=black&white, 28 x 28

train1_norm=BatchNormalization(axis=-1, name='t1n')(input_img)
train1_c1 = Conv2D(16, (3, 3), activation='relu', padding='same', name='t1c1')(train1_norm)
train1_decoded = Conv2DTranspose(1, (3, 3), activation='relu', padding='same', name='t1tc1')(train1_c1)

train1_autoencoder = Model(input_img, train1_decoded)
train1_autoencoder.compile(optimizer='adadelta', loss=own_loss)

train1_autoencoder.fit(x_9, x_9, epochs=10, batch_size=20,
               shuffle=True, validation_data=(x_9_test, x_9_test), verbose=1)

Train on 5000 samples, validate on 500 samples
Epoch 1/10
5000/5000 [==============================] - 4s 760us/step - loss: 0.0879 - val_loss: -0.0075
Epoch 2/10
5000/5000 [==============================] - 4s 714us/step - loss: -0.0084 - val_loss: -0.0100
Epoch 3/10
5000/5000 [==============================] - 4s 713us/step - loss: -0.0098 - val_loss: -0.0109
Epoch 4/10
5000/5000 [==============================] - 4s 736us/step - loss: -0.0104 - val_loss: -0.0101
Epoch 5/10
5000/5000 [==============================] - 4s 716us/step - loss: -0.0107 - val_loss: -0.0096
Epoch 6/10
5000/5000 [==============================] - 4s 717us/step - loss: -0.0110 - val_loss: -0.0116
Epoch 7/10
5000/5000 [==============================] - 4s 717us/step - loss: -0.0111 - val_loss: -0.0117
Epoch 8/10
5000/5000 [==============================] - 4s 745us/step - loss: -0.0112 - val_loss: -0.0117
Epoch 9/10
5000/5000 [==============================] - 4s 721us/step - loss: -0.0113 - val_loss: -0.0115


In [4]:
### pretrain2

creator_train2 = Model(input_img, train1_c1)

a = creator_train2.get_weights()

creator_train2.get_layer("t1c1").set_weights(train1_autoencoder.get_layer('t1c1').get_weights())

b = creator_train2.get_weights()


for i in range(len(a)):
    for j in range(len(a[0])):
        print(a[i][j]==b[i][j])

        
'''
test_input_img = Input(shape=(28, 28,1)) # 1ch=black&white, 28 x 28

test_train1_norm=BatchNormalization(axis=-1, name='t1n')(input_img)
test_train1_c1 = Conv2D(16, (3, 3), activation='relu', padding='same', name='t1c1',
                        kernel_initializer=train1_autoencoder.get_layer('t1c1').get_weights()[0],
                        bias_initializer=train1_autoencoder.get_layer('t1c1').get_weights()[1])(train1_norm)
test_train1_decoded = Conv2DTranspose(1, (3, 3), activation='relu', padding='same', name='t1tc1')(train1_c1)

test_train1_autoencoder = Model(input_img, train1_decoded)
'''



creator_train2 = Model(input_img, train1_c1)
creator_train2.get_layer("t1c1").set_weights(train1_autoencoder.get_layer('t1c1').get_weights())

input_train2 = creator_train2.predict(x_9)

print(input_train2.shape)

#---

True
True
True
True
[[[ True  True  True  True  True  True  True  True  True  True  True
    True  True  True  True  True]]

 [[ True  True  True  True  True  True  True  True  True  True  True
    True  True  True  True  True]]

 [[ True  True  True  True  True  True  True  True  True  True  True
    True  True  True  True  True]]]
True
(5000, 28, 28, 16)


In [ ]:
input_img_train2 = Input(shape=(28, 28, 16))
train2_c1 = Conv2D(8, (3, 3), activation='relu', padding='same', name='t2c1')(input_img_train2)
train2_decoded = Conv2DTranspose(16, (3, 3), activation='relu', padding='same', name='t2tc1')(train2_c1)

train2_autoencoder = Model(input_img_train2, train2_decoded)
train2_autoencoder.compile(optimizer='adadelta', loss=own_loss)

train2_autoencoder.fit(input_train2, input_train2, epochs=10, batch_size=20, verbose=1)

### pretrain3

creator_train3 = Model(input_img_train2, train2_c1)
creator_train3.get_layer("t2c1").set_weights(train2_autoencoder.get_layer('t2c1').get_weights())

input_train3 = creator_train3.predict(input_train2)

print(input_train2.shape)

#---

input_img_train3 = Input(shape=(28, 28, 8))
train3_p1 = MaxPooling2D(pool_size=(2, 2), name='t3p1')(input_img_train3)
train3_c1 = Conv2D(4, (3, 3), activation='relu', padding='same', name='t3c1')(train3_p1)
train3_tc1 = Conv2DTranspose(8, (3, 3), activation='relu', padding='same', name='t3tc1')(train3_c1)
train3_decoded = UpSampling2D(size=(2, 2), name='t3u1')(train3_tc1)

train3_autoencoder = Model(input_img_train3, train3_decoded)
train3_autoencoder.compile(optimizer='adadelta', loss=own_loss)

train3_autoencoder.fit(input_train3, input_train3, epochs=10, batch_size=20, verbose=1)

### unroll

input_img = Input(shape=(28, 28,1)) # 1ch=black&white, 28 x 28

norm=BatchNormalization(axis=-1, name='n')(input_img)
c1 = Conv2D(16, (3, 3), activation='relu', padding='same', name='c1')(norm)
c2 = Conv2D(8, (3, 3), activation='relu', padding='same', name='c2')(c1)
p1 = MaxPooling2D(pool_size=(2, 2), name='p1')(c2)
encoded = Conv2D(4, (3, 3), activation='relu', padding='same', name='c3')(p1)
tc3 = Conv2DTranspose(8, (3, 3), activation='relu', padding='same', name='tc3')(encoded)
u1 = UpSampling2D(size=(2, 2), name='u1')(tc3)
tc2 = Conv2DTranspose(16, (3, 3), activation='relu', padding='same', name='tc2')(u1)
decoded = Conv2DTranspose(1, (3, 3), activation='relu', padding='same', name='tc1')(tc2)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss=own_loss)

autoencoder.get_layer("c1").set_weights(train1_autoencoder.get_layer('t1c1').get_weights())
autoencoder.get_layer("c2").set_weights(train2_autoencoder.get_layer('t2c1').get_weights())
autoencoder.get_layer("c3").set_weights(train3_autoencoder.get_layer('t3c1').get_weights())
autoencoder.get_layer("tc3").set_weights(train3_autoencoder.get_layer('t3tc1').get_weights())
autoencoder.get_layer("tc2").set_weights(train2_autoencoder.get_layer('t2tc1').get_weights())
autoencoder.get_layer("tc1").set_weights(train1_autoencoder.get_layer('t1tc1').get_weights())

### finetune

autoencoder.fit(x_9, x_9, epochs=10, batch_size=20,
            shuffle=True, validation_data=(x_9_test, x_9_test), verbose=1)

### plot

decoded_imgs = autoencoder.predict(x_9_test)
show_imgs(x_9_test,decoded_imgs)

print(autoencoder.summary())

In [ ]:
enbig1 = UpSampling2D(size=(2, 2))(encoded)
#enbig2 = UpSampling2D(size=(2, 2))(enbig1)

autoencoder_show_hidden = Model(input_img, enbig1)
encoded_imgs = autoencoder_show_hidden.predict(x_9_test)

def show_hidden(x_test, encoded_imgs, n=10):
    
    en0=np.zeros((encoded_imgs.shape[0],encoded_imgs.shape[1],encoded_imgs.shape[2]))
    en1=np.zeros((encoded_imgs.shape[0],encoded_imgs.shape[1],encoded_imgs.shape[2]))
    en2=np.zeros((encoded_imgs.shape[0],encoded_imgs.shape[1],encoded_imgs.shape[2]))
    en3=np.zeros((encoded_imgs.shape[0],encoded_imgs.shape[1],encoded_imgs.shape[2]))

    for i in range(encoded_imgs.shape[0]):
        for j in range(encoded_imgs.shape[1]):
            for k in range(encoded_imgs.shape[2]):
                en0[i][j][k]=encoded_imgs[i][j][k][0]
                en1[i][j][k]=encoded_imgs[i][j][k][1]
                en2[i][j][k]=encoded_imgs[i][j][k][2]
                en3[i][j][k]=encoded_imgs[i][j][k][3]
    
    plt.figure(figsize=(20, 4))
    for i in range(n):
        ax = plt.subplot(5, n, i+1)
        plt.imshow(x_test[i].reshape(28,28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        if decoded_imgs is not None:
            ax = plt.subplot(5, n, i+ 1 +n)
            plt.imshow(en0[i].reshape(28,28))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            
            ax = plt.subplot(5, n, i+ 1 +2*n)
            plt.imshow(en1[i].reshape(28,28))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            
            ax = plt.subplot(5, n, i+ 1 +3*n)
            plt.imshow(en2[i].reshape(28,28))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            
            ax = plt.subplot(5, n, i+ 1 +4*n)
            plt.imshow(en3[i].reshape(28,28))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
    plt.show()

show_hidden(x_9_test, encoded_imgs)